# Model Training and Evaluation

This notebook covers the definition, training, and evaluation of a transformer-based multi-label text classification model.

## 1. Load Prepared Datasets and Libraries

Load the PyTorch datasets and import required libraries for model training and evaluation.

In [ ]:
# %pip install mlflow

  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached gunicorn-23.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached gunicorn-23.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached sqlalchemy-2.0.43-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.6 kB)
  Using cached sqlalchemy-2.0.43-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.6 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
  Using cached google_auth-2.4

In [ ]:
# %pip install databricks-sdk

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Import libraries and load datasets
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
import numpy as np
from mlflow.models import infer_signature

In [16]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
# Set Databricks credentials from environment variables (do not hardcode in code!)
import os
import mlflow
DATABRICKS_HOST = os.getenv("DATABRICKS_HOST")
DATABRICKS_TOKEN = os.getenv("DATABRICKS_TOKEN")
if not DATABRICKS_HOST or not DATABRICKS_TOKEN:
    raise ValueError("DATABRICKS_HOST and DATABRICKS_TOKEN must be set in your environment.\n\nIn Colab, use:\n%env DATABRICKS_HOST=https://<your-databricks-instance>\n%env DATABRICKS_TOKEN=<your-token>\n\nLocally, set them in your shell or .env file.")
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/j2damax@gmail.com/serendip-travel-review-classifier-experiments")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/3866477269740015', creation_time=1758424394648, experiment_id='3866477269740015', last_update_time=1758427431013, lifecycle_stage='active', name='/Users/j2damax@gmail.com/serendip-travel-review-classifier-experiments', tags={'mlflow.experiment.sourceName': '/Users/j2damax@gmail.com/serendip-travel-review-classifier-experiments',
 'mlflow.experimentKind': 'genai_development',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'j2damax@gmail.com',
 'mlflow.ownerId': '5804221812504751'}>

In [10]:
# Define ReviewsDataset class (must match the one used in 03_modeling.ipynb)
from torch.utils.data import Dataset
class ReviewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels, dtype=torch.float32)
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
    def __len__(self):
        return len(self.labels)

In [13]:
#train_data = torch.load('../data/processed/train_dataset.pt', weights_only=False)
#test_data = torch.load('../data/processed/test_dataset.pt', weights_only=False)

# Load datasets from Google Drive
train_data = torch.load('/content/drive/MyDrive/SerendipTravel/data/processed/train_dataset.pt', weights_only=False)
test_data = torch.load('/content/drive/MyDrive/SerendipTravel/data/processed/test_dataset.pt', weights_only=False)

# Reuse your ReviewsDataset class definition here
train_dataset = ReviewsDataset(train_data['encodings'], train_data['labels'])
test_dataset = ReviewsDataset(test_data['encodings'], test_data['labels'])

## 2. Model Definition and Configuration

Define the transformer model for multi-label classification and set up optimizer, loss, and training parameters.

In [14]:
# Model definition and configuration
num_labels = train_data['labels'].shape[1]
print(f"Number of labels: {num_labels}")

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=num_labels,
    problem_type='multi_label_classification'
)

# Training parameters
batch_size = 16
epochs = 3
learning_rate = 2e-5

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

# Loss function for multi-label
loss_fn = torch.nn.BCEWithLogitsLoss()

Number of labels: 4


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 3. Model Training and Evaluation

Train the model and evaluate its performance on the test set.

In [20]:
# Training and evaluation loop with MLflow logging
import mlflow.pytorch
with mlflow.start_run(run_name="bert-multilabel-baseline"):
    # Log hyperparameters
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("model_name", "bert-base-uncased")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
        avg_train_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} - Training loss: {avg_train_loss:.4f}")
        mlflow.log_metric("train_loss", avg_train_loss, step=epoch)

    # Evaluation
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].cpu().numpy()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits.cpu().numpy()
            preds = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
            all_preds.append(preds)
            all_labels.append(labels)

    import numpy as np
    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    # Compute metrics
    from sklearn.metrics import accuracy_score, f1_score
    test_accuracy = accuracy_score(all_labels, all_preds)
    test_macro_f1 = f1_score(all_labels, all_preds, average='macro')
    print("Test accuracy:", test_accuracy)
    print("Test macro F1:", test_macro_f1)
    mlflow.log_metric("test_accuracy", test_accuracy)
    mlflow.log_metric("test_macro_f1", test_macro_f1)

    # Prepare a sample input and output
    example_batch = next(iter(test_loader))
    inputs = {
        "input_ids": example_batch["input_ids"][:1].cpu().numpy(),
        "attention_mask": example_batch["attention_mask"][:1].cpu().numpy()
    }
    outputs = model(
        input_ids=example_batch["input_ids"][:1].to(device),
        attention_mask=example_batch["attention_mask"][:1].to(device)
    ).logits.cpu().detach().numpy()

    signature = infer_signature(inputs, outputs)

    mlflow.pytorch.log_model(
        model,
        name="model",
        signature=signature,
        pip_requirements=["torch==2.8.0+cu126", "torchvision==0.23.0+cu126"]
    )

    np.save("predictions.npy", all_preds)
    mlflow.log_artifact("predictions.npy")

Epoch 1/3 - Training loss: 0.0573
Epoch 2/3 - Training loss: 0.0573
Epoch 3/3 - Training loss: 0.0578
Test accuracy: 0.9245049504950495
Test macro F1: 0.9298934093407507
🏃 View run bert-multilabel-baseline at: https://dbc-cfeb31c8-2841.cloud.databricks.com/ml/experiments/3866477269740015/runs/5728587524314f69b56434d8d58e9490
🧪 View experiment at: https://dbc-cfeb31c8-2841.cloud.databricks.com/ml/experiments/3866477269740015
